In [2]:
import datetime as dt
import yfinance as yf
import plotly.graph_objects as go
import pandas as pd
import mplfinance as mpf
import numpy as np
import math
import plotly.io as pio
from plotly.subplots import make_subplots
pio.renderers.default = 'notebook_connected'


In [3]:


start = dt.datetime(2015, 1, 1)
end = dt.datetime.now()


df = yf.download('CBA.AX', start=start, end=end)


[*********************100%***********************]  1 of 1 completed


In [4]:

df['MA50'] = df['Close'].rolling(window=50, min_periods=0).mean()
df['MA200'] = df['Close'].rolling(window=200, min_periods=0).mean()

df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

df = df.dropna(subset=['Open', 'High', 'Low', 'Close', 'Volume'])

In [5]:

fig=make_subplots(
    rows=2, cols=1, 
    shared_xaxes=True, 
    row_heights=[0.7, 0.3],  
    vertical_spacing=0.05 
)


In [6]:
fig.add_trace(
    go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Candlestick'
    ),
    row=1, col=1
)


fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['MA50'],
        mode='lines',
        line=dict(color='blue', width=1.5),
        name='MA50'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['MA200'],
        mode='lines',
        line=dict(color='red', width=1.5),
        name='MA200'
    ),
    row=1, col=1
)


fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['Volume'],
        mode='lines',
        line=dict(color='purple', width=1.5),
        name='Volume'
    ),
    row=2, col=1
)


fig.update_layout(
    title='CBA.AX Candlestick Chart with Moving Averages and Volume',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis2_title='Date', 
    yaxis2_title='Volume',
    template='plotly_dark',
    height=900,  
    showlegend=True
)


fig.show()


In [117]:
#stocks =["LUPIN.NS","VOLTAS.NS","NMDC.NS","ZYDUSLIFE.NS","SAIL.NS"]

start_date = '2018-01-01'
end_date = '2023-12-31'

df_benchmark = yf.download('LUPIN.NS',start=start_date,end=end_date)

investment_value =100000

shares_bought = investment_value/df_benchmark['Close'].iloc[0]

df_benchmark['Portfolio Value'] = shares_bought*df_benchmark['Close']

final_portfolio_value = df_benchmark['Portfolio Value'].iloc[-1]

benchmark_returns = (final_portfolio_value-investment_value)*100/investment_value

print("Benchmark Returns (Buy and Hold returns of the stock): ",benchmark_returns,"%")

s= df_benchmark['Portfolio Value']
s=np.log(s/s.shift(1))
s=s[1:]
ra=s.mean()*252
sigma= s.std()*math.sqrt(252)
rb=0.07
sharpe=(ra-rb)/sigma
print("Benchmark sharpe ratio: ",sharpe)



[*********************100%***********************]  1 of 1 completed

Benchmark Returns (Buy and Hold returns of the stock):  49.782049382606836 %
Benchmark sharpe ratio:  -0.003978349735167335


In [116]:
df_indicator =yf.download('LUPIN.NS',start=start_date,end=end_date)

intial_capital = 100000 #INR
capital = intial_capital   
shares=0
position=0 #1 if I'm holding the stock, 0 if otherwise
portfolio =[]
df_indicator.columns = [col[0] if isinstance(col, tuple) else col for col in df_indicator.columns]
#print(df_indicator.columns)

df_indicator['SMA50'] = df_indicator['Close'].rolling(window=50,min_periods=0).mean()
df_indicator['SMA200'] = df_indicator['Close'].rolling(window=200,min_periods=0).mean()

df_indicator['Midpoint'] = (df_indicator['Open']+df_indicator['Close'])/2
df_indicator['High Range'] = df_indicator['High']-df_indicator['Midpoint']
df_indicator['Low Range'] = df_indicator['Midpoint']-df_indicator['Low']
df_indicator['High Range Volume'] = df_indicator['Volume']*(df_indicator['High Range']/(df_indicator['High Range']+df_indicator['Low Range']))
df_indicator['Low Range Volume'] = df_indicator['Volume']*(df_indicator['Low Range']/(df_indicator['High Range']+df_indicator['Low Range']))
df_indicator['HLVR'] = df_indicator['High Range Volume']/df_indicator['Low Range Volume']
#df_indicator['HLVR'] = df_indicator['HLVR'].rolling(window=200,min_periods = 0).mean()

for i in range(len(df_indicator)):
    if (df_indicator['HLVR'].iloc[i]>1.5 and position==0 and df_indicator['SMA50'].iloc[i] > df_indicator['SMA200'].iloc[i]):
        position=1
        shares=capital//df_indicator['Close'].iloc[i]
        capital-=shares*df_indicator['Close'].iloc[i]
    elif (df_indicator['HLVR'].iloc[i]<0 and position==1 and df_indicator['SMA200'].iloc[i]>df_indicator['SMA50'].iloc[i]):
        position=0
        shares=0
        capital+=shares*df_indicator['Close'].iloc[i]
    portfolio_value=  capital + shares*df_indicator['Close'].iloc[i]
    portfolio.append(portfolio_value)



final_capital = portfolio[-1]
total_return = (final_capital-intial_capital)*100/intial_capital

print("Final Capital: ",final_capital)
print("Total Return: ", total_return,"%")
#print("Sharpe Ratio: ",indicator_sharpe)

[*********************100%***********************]  1 of 1 completed

Final Capital:  160844.84704589844
Total Return:  60.84484704589844 %
